In [ ]:
# for plotting

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

sns.set()
sns.set_context('poster')
sns.set_style('ticks')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'STIXGeneral'
plt.rcParams["mathtext.fontset"] = 'stix'

In [ ]:
import numpy as np
import xarray as xr
import metpy
from metpy.units import units

These are quicklook plots to evaluate the profiles and large-scale horizontal advective tendencies of $\theta$ and $r_v$, and the large-scale subsidence in the forcing datasets.

---

In [ ]:
h = 30. # surface elevation of ENA site in m
lat = 39.0916 # deg N
lon = -28.0257 # deg E
dx = 2.5 # deg, half-width of averaging boxes for omega etc.

In [ ]:
obs_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/obs'
forc_dir = '/ccsopen/home/h1x/scratch/ena_forcing_check/forcing'

In [ ]:
# Code from ECWMF for calculating geopotential on model levels

# # compute moist temperature
# t_level = t_level * (1. + 0.609133 * q_level)

# # compute the pressures (on half-levels)
# ph_lev, ph_levplusone = get_ph_levs(values, lev)

# if lev == 1:
#     dlog_p = np.log(ph_levplusone / 0.1)
#     alpha = np.log(2)
# else:
#     dlog_p = np.log(ph_levplusone / ph_lev)
#     alpha = 1. - ((ph_lev / (ph_levplusone - ph_lev)) * dlog_p)

# t_level = t_level * R_D

# # z_f is the geopotential of this full level
# # integrate from previous (lower) half-level z_h to the
# # full level
# z_f = z_h + (t_level * alpha)

# # z_h is the geopotential of 'half-levels'
# # integrate z_h to next half level
# z_h = z_h + (t_level * dlog_p)

# return z_h, z_f

In [ ]:
def calc_tp_era5(data):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    v = np.genfromtxt('era5_table.csv', delimiter=',', skip_header=1, missing_values='-')
    a = v[:,1]
    b = v[:,2]
    t = data.t
    lnsp = data.lnsp.values[:,0,:,:]
    nt, nz, ny, nx = t.shape
    pi = np.zeros((nt, nz+1, ny, nx))
    pi[:] = np.exp(lnsp[:,np.newaxis,:,:])
    pi = a[np.newaxis,:,np.newaxis, np.newaxis]+pi*b[np.newaxis,:,np.newaxis, np.newaxis]
    p = (pi[:,1:,:,:] + pi[:,:-1,:,:])*0.5
    tpv = metpy.calc.potential_temperature(p*units.pascal, t)
    tp = t.copy(deep=True, data=tpv)
    tp.rename('Air Potential Temperature')
    return tp

def extract_era5_profiles(data, time):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    v = np.genfromtxt('era5_table.csv', delimiter=',', skip_header=1, missing_values='-')
    a = v[:,1]
    b = v[:,2]
    t = data.t.loc[time].values
    qv = data.q.loc[time].values
    lnsp = data.lnsp.loc[time].values[0]
    # print(lnsp)
    sgp = data.z.loc[time].values[0]
    # print(sgp)
    nz = t.shape[0]
    pi = np.zeros(nz+1)
    pi[:] = np.exp(lnsp)
    pi = a+pi*b
    # print(pi[-1], pi[0])
    p = (pi[1:] + pi[:-1])*0.5
    # Pa, only used for geopotential calculation below
    pi[0] = 0.1
    tm = t*(1.0+0.609133*qv)*units.kelvin
    dpi = pi[1:] - pi[:-1]
    dlnpi = np.log(pi[1:]/pi[:-1])
    # have not got time to derive alpha, this is just what is given in the ERA5 documentation
    alpha = 1.0 - dlnpi*pi[:-1]/dpi
    # following ERA5 doc
    alpha[0] = np.log(2.)
    dph = rd*tm*dlnpi
    ph = np.zeros(nz+1)
    ph[:-1][::-1] = np.cumsum(dph[::-1])
    ph[:] = ph[:] + sgp
    zi = ph*units('m**2/s**2')/g
    z = zi[1:] + rd*tm*alpha/g
    tp = metpy.calc.potential_temperature(p*units.pascal, t*units.kelvin)
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)
    return z, p*units.pascal, tp, rv

In [ ]:
def extract_merra2_profiles_ana(data, time, sgp=196.70242):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    ptop = data.ptop
    dpi = data.DELP.loc[time] 
    nz = dpi.shape[0]
    t = data.T.loc[time].metpy.quantify()
    qv = data.QV.loc[time].metpy.quantify()
    tm = t*(1.0+0.609133*qv)
    pi = np.zeros(nz+1)
    pi[1:] = np.cumsum(dpi) + ptop
    pi[0] = ptop
    p = (pi[1:] + pi[:-1])*0.5
    dlnpi = np.log(pi[1:]/pi[:-1])
    alpha = 1.0 - dlnpi*pi[:-1]/dpi
    dph = rd*tm*dlnpi
    ph = np.zeros(nz+1)
    ph[:-1][::-1] = np.cumsum(dph[::-1])
    ph[:] = ph[:] + sgp
    zi = ph*units('m**2/s**2')/g
    z = zi[1:] + rd*tm*alpha/g
    tp = metpy.calc.potential_temperature(p*units.pascal, t)
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)
    return z, p*units.pascal, tp, rv

def extract_merra2_profiles_asm(data, time, sgp=196.70242):
    rd = metpy.constants.dry_air_gas_constant
    g = metpy.constants.earth_gravity
    z = data.H.loc[time].metpy.quantify()
    t = data.T.loc[time].metpy.quantify()
    qv = data.QV.loc[time].metpy.quantify()
    p = data.PL.loc[time].metpy.quantify()
    tp = metpy.calc.potential_temperature(p, t)
    rv = metpy.calc.mixing_ratio_from_specific_humidity(qv)
    return z, p, tp, rv

In [ ]:
armbe = xr.open_dataset(f'{obs_dir}/enaarmbeatmC1.c1.20180101.003000.nc')
armbecld = xr.open_dataset(f'{obs_dir}/enaarmbecldradC1.c1.20180101.003000.nc')

In [ ]:
time_era5 = np.arange(0, 24.1, 1.)
era5_sfc = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180206-20180208-sfc.nc')
era5_l = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180206-20180208-ml.nc')
era5_sfc_ena = era5_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
era5_l_ena = era5_l.sel(longitude=lon+360., latitude=lat, method='nearest')
era5_sfc_ena = era5_sfc_ena.sortby(era5_sfc_ena.time)

In [ ]:
time_fnl = np.arange(0, 21.1, 3.)
fnl_sfc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_surface.nc')
fnl_sfc2 = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_heightAboveGround.nc')
fnl_l = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180207_fc_isobaricInhPa.nc')
fnl_sfc_ena = fnl_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_sfc2_ena = fnl_sfc2.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_l_ena = fnl_l.sel(longitude=lon+360., latitude=lat, method='nearest')

In [ ]:
time_merra2 = np.arange(0.5, 23.7, 1.0)
merra2_l = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.20180207.nc4')
merra2_l_ana = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst6_3d_ana_Nv.20180207.nc4')
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180207.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180207.nc4')
merra2_l_ena = merra2_l.sel(lon=lon, lat=lat, method='nearest')
merra2_l_ena_ana = merra2_l_ana.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc_ena = merra2_sfc.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc2_ena = merra2_sfc2.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
tp = calc_tp_era5(era5_l) 
ma_era5 = metpy.calc.advection(era5_l.q, era5_l.u, era5_l.v).loc[:,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx].mean(axis=(2,3))
ta_era5 = metpy.calc.advection(tp, era5_l.u, era5_l.v).loc[:,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx].mean(axis=(2,3))
w_era5 = era5_l.w.loc[:,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx].mean(axis=(2,3))

In [ ]:
t = fnl_l.t.loc[:,:,lat+10.:lat-10., lon+360.-10.:lon+360.+10.]
rh = fnl_l.r.loc[:,:,lat+10.:lat-10., lon+360.-10.:lon+360.+10.]
p = np.zeros(t.shape)
p[:] = fnl_l.level.values[np.newaxis,:,np.newaxis,np.newaxis]
tp = metpy.calc.potential_temperature(p*units.hectopascal, t)
rv = metpy.calc.mixing_ratio_from_relative_humidity(p*units.hectopascal, t, rh)
w_fnl = fnl_l.w.loc[:,:,lat+dx:lat-dx, lon+360.-dx:lon+360+dx].mean(axis=(2,3))
ma_fnl = w_fnl.copy(deep=True, data=metpy.calc.advection(rv, fnl_l.u.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10], fnl_l.v.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10]).loc[:,:,lat+dx:lat-dx, lon+360.-dx:lon+360+dx].mean(axis=(2,3)))
ta_fnl = w_fnl.copy(deep=True, data=metpy.calc.advection(tp, fnl_l.u.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10], fnl_l.v.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10]).loc[:,:,lat+dx:lat-dx, lon+360.-dx:lon+360+dx].mean(axis=(2,3)))

In [ ]:
ma_merra2 = metpy.calc.advection(merra2_l.QV.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.U.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.V.loc[:,:,lat-10:lat+10, lon-10:lon+10]).loc[:,:,lat-dx:lat+dx,lon-dx:lon+dx].mean(axis=(2,3))
tp = metpy.calc.potential_temperature(merra2_l.PL.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.T.loc[:,:,lat-10:lat+10, lon-10:lon+10])
ta_merra2 = metpy.calc.advection(tp, merra2_l.U.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.V.loc[:,:,lat-10:lat+10, lon-10:lon+10]).loc[:,:,lat-dx:lat+dx,lon-dx:lon+dx].mean(axis=(2,3))
w_merra2 = merra2_l.OMEGA.loc[:,:,lat-dx:lat+dx,lon-dx:lon+dx].mean(axis=(2,3))

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# SONDE 1200 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180207.113000.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', label='SONDE 1130')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', label='SONDE 1130')
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-07T12:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
tp = np.compress(lim, tp_era5)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
rv = np.compress(lim, rv_era5)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
# FNL
z = fnl_l_ena.gh.loc['2018-02-07T12:00:00']
t = fnl_l_ena.t.loc['2018-02-07T12:00:00']
rh = fnl_l_ena.r.loc['2018-02-07T12:00:00']
p = fnl_l_ena.level
tp = metpy.calc.potential_temperature(p*units.hectopascal, t.metpy.quantify())
rv = metpy.calc.mixing_ratio_from_relative_humidity(p*units.hectopascal, t.metpy.quantify(), rh.metpy.quantify())
lim = z < 5000.
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
# SONDE 1800 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180207.174200.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', label='SONDE 1742')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', label='SONDE 1742')
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-07T18:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
tp = np.compress(lim, tp_era5)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
rv = np.compress(lim, rv_era5)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
# FNL
z = fnl_l_ena.gh.loc['2018-02-07T18:00:00']
t = fnl_l_ena.t.loc['2018-02-07T18:00:00']
rh = fnl_l_ena.r.loc['2018-02-07T18:00:00']
p = fnl_l_ena.level
tp = metpy.calc.potential_temperature(p*units.hectopascal, t.metpy.quantify())
rv = metpy.calc.mixing_ratio_from_relative_humidity(p*units.hectopascal, t.metpy.quantify(), rh.metpy.quantify())
lim = z < 5000.
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\theta$ (K)')
ax1.set_title('2018-02-07')
# ax2.ylabel('Altitude (km)')
ax2.set_ylabel('')
ax2.set_ylim(0, 5)
ax2.set_xlabel(r'$r_v$ (g/kg)')
ax2.set_title('2018-02-07')
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# SONDE 1200 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180207.113000.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', label='SONDE 1130')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', label='SONDE 1130')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_ana(merra2_l_ena_ana, '2018-02-07T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1200', fillstyle='none')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1200', fillstyle='none')
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1200')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1200')
# SONDE 1800 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180207.174200.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', label='SONDE 1742')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', label='SONDE 1742')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_ana(merra2_l_ena_ana, '2018-02-07T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1800', fillstyle='none')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1800', fillstyle='none')
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1800')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\theta$ (K)')
ax1.set_title('2018-02-07')
# ax2.ylabel('Altitude (km)')
ax2.set_ylabel('')
ax2.set_ylim(0, 5)
ax2.set_xlabel(r'$r_v$ (g/kg)')
ax2.set_title('2018-02-07')
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
fig = plt.figure(figsize=(6, 9))
ax1 = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# 1200 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-07T12:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
w = np.compress(lim, w_era5.loc['2018-02-07T12:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
# FNL
z = fnl_l_ena.gh.loc['2018-02-07T12:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
w = np.compress(lim, w_fnl.loc['2018-02-07T12:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
w = np.compress(lim, w_merra2.loc['2018-02-07T12:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
# 1800 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-07T18:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
w = np.compress(lim, w_era5.loc['2018-02-07T18:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
# FNL
z = fnl_l_ena.gh.loc['2018-02-07T18:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
w = np.compress(lim, w_fnl.loc['2018-02-07T18:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
w = np.compress(lim, w_merra2.loc['2018-02-07T18:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\omega$ (hPa/hr)')
ax1.set_title('2018-02-07')
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# 1200 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-07T12:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
ta = np.compress(lim, ta_era5.loc['2018-02-07T12:00:00'])
ma = np.compress(lim, ma_era5.loc['2018-02-07T12:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
# FNL
z = fnl_l_ena.gh.loc['2018-02-07T12:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_fnl.loc['2018-02-07T12:00:00'])
ma = np.compress(lim, ma_fnl.loc['2018-02-07T12:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_merra2.loc['2018-02-07T12:00:00'])
ma = np.compress(lim, ma_merra2.loc['2018-02-07T12:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
# 1800 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-07T18:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
ta = np.compress(lim, ta_era5.loc['2018-02-07T18:00:00'])
ma = np.compress(lim, ma_era5.loc['2018-02-07T18:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
# FNL
z = fnl_l_ena.gh.loc['2018-02-07T18:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_fnl.loc['2018-02-07T18:00:00'])
ma = np.compress(lim, ma_fnl.loc['2018-02-07T18:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-07T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_merra2.loc['2018-02-07T18:00:00'])
ma = np.compress(lim, ma_merra2.loc['2018-02-07T18:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\theta$ horizontal advection (K/hr)')
ax1.set_title('2018-02-07')
ax2.set_ylabel('')
ax2.set_ylim(0, 5)
ax2.set_xlabel(r'$r_v$ horizontal advection (g/kg/hr)')
ax2.set_title('2018-02-07')
plt.legend(fontsize=12)
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
era5_sfc = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180215-20180217-sfc.nc')
era5_l = xr.open_dataset(f'{forc_dir}/era5/data/2018/ERA5-20180215-20180217-ml.nc')
era5_sfc_ena = era5_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
era5_l_ena = era5_l.sel(longitude=lon+360., latitude=lat, method='nearest')
era5_sfc_ena = era5_sfc_ena.sortby(era5_sfc_ena.time)

In [ ]:
time_fnl = np.arange(0, 21.1, 3.)
fnl_sfc = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_surface.nc')
fnl_sfc2 = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_heightAboveGround.nc')
fnl_l = xr.open_dataset(f'{forc_dir}/fnl0.25deg/data/2018/kwbc_20180216_fc_isobaricInhPa.nc')
fnl_sfc_ena = fnl_sfc.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_sfc2_ena = fnl_sfc2.sel(longitude=lon+360., latitude=lat, method='nearest')
fnl_l_ena = fnl_l.sel(longitude=lon+360., latitude=lat, method='nearest')

In [ ]:
time_merra2 = np.arange(0.5, 23.7, 1.0)
merra2_l = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst3_3d_asm_Nv.20180216.nc4')
merra2_l_ana = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.inst6_3d_ana_Nv.20180216.nc4')
merra2_sfc = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_flx_Nx.20180216.nc4')
merra2_sfc2 = xr.open_dataset(f'{forc_dir}/merra2/data/MERRA2_400.tavg1_2d_slv_Nx.20180216.nc4')
merra2_l_ena = merra2_l.sel(lon=lon, lat=lat, method='nearest')
merra2_l_ena_ana = merra2_l_ana.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc_ena = merra2_sfc.sel(lon=lon, lat=lat, method='nearest')
merra2_sfc2_ena = merra2_sfc2.sel(lon=lon, lat=lat, method='nearest')

In [ ]:
tp = calc_tp_era5(era5_l) 
ma_era5 = metpy.calc.advection(era5_l.q, era5_l.u, era5_l.v).loc[:,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx].mean(axis=(2,3))
ta_era5 = metpy.calc.advection(tp, era5_l.u, era5_l.v).loc[:,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx].mean(axis=(2,3))
w_era5 = era5_l.w.loc[:,:,lat+dx:lat-dx, lon+360-dx:lon+360+dx].mean(axis=(2,3))

In [ ]:
t = fnl_l.t.loc[:,:,lat+10.:lat-10., lon+360.-10.:lon+360.+10.]
rh = fnl_l.r.loc[:,:,lat+10.:lat-10., lon+360.-10.:lon+360.+10.]
p = np.zeros(t.shape)
p[:] = fnl_l.level.values[np.newaxis,:,np.newaxis,np.newaxis]
tp = metpy.calc.potential_temperature(p*units.hectopascal, t)
rv = metpy.calc.mixing_ratio_from_relative_humidity(p*units.hectopascal, t, rh)
w_fnl = fnl_l.w.loc[:,:,lat+dx:lat-dx, lon+360.-dx:lon+360+dx].mean(axis=(2,3))
ma_fnl = w_fnl.copy(deep=True, data=metpy.calc.advection(rv, fnl_l.u.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10], fnl_l.v.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10]).loc[:,:,lat+dx:lat-dx, lon+360.-dx:lon+360+dx].mean(axis=(2,3)))
ta_fnl = w_fnl.copy(deep=True, data=metpy.calc.advection(tp, fnl_l.u.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10], fnl_l.v.loc[:,:,lat+10:lat-10, lon+360.-10:lon+360+10]).loc[:,:,lat+dx:lat-dx, lon+360.-dx:lon+360+dx].mean(axis=(2,3)))

In [ ]:
ma_merra2 = metpy.calc.advection(merra2_l.QV.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.U.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.V.loc[:,:,lat-10:lat+10, lon-10:lon+10]).loc[:,:,lat-dx:lat+dx,lon-dx:lon+dx].mean(axis=(2,3))
tp = metpy.calc.potential_temperature(merra2_l.PL.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.T.loc[:,:,lat-10:lat+10, lon-10:lon+10])
ta_merra2 = metpy.calc.advection(tp, merra2_l.U.loc[:,:,lat-10:lat+10, lon-10:lon+10], merra2_l.V.loc[:,:,lat-10:lat+10, lon-10:lon+10]).loc[:,:,lat-dx:lat+dx,lon-dx:lon+dx].mean(axis=(2,3))
w_merra2 = merra2_l.OMEGA.loc[:,:,lat-dx:lat+dx,lon-dx:lon+dx].mean(axis=(2,3))

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# SONDE 1200 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180216.113100.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', label='SONDE 1131')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', label='SONDE 1131')
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-16T12:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
tp = np.compress(lim, tp_era5)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
rv = np.compress(lim, rv_era5)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
# FNL
z = fnl_l_ena.gh.loc['2018-02-16T12:00:00']
t = fnl_l_ena.t.loc['2018-02-16T12:00:00']
rh = fnl_l_ena.r.loc['2018-02-16T12:00:00']
p = fnl_l_ena.level
tp = metpy.calc.potential_temperature(p*units.hectopascal, t.metpy.quantify())
rv = metpy.calc.mixing_ratio_from_relative_humidity(p*units.hectopascal, t.metpy.quantify(), rh.metpy.quantify())
lim = z < 5000.
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
# SONDE 1800 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180216.173000.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', label='SONDE 1730')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', label='SONDE 1730')
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-16T18:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
tp = np.compress(lim, tp_era5)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
rv = np.compress(lim, rv_era5)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
# FNL
z = fnl_l_ena.gh.loc['2018-02-16T18:00:00']
t = fnl_l_ena.t.loc['2018-02-16T18:00:00']
rh = fnl_l_ena.r.loc['2018-02-16T18:00:00']
p = fnl_l_ena.level
tp = metpy.calc.potential_temperature(p*units.hectopascal, t.metpy.quantify())
rv = metpy.calc.mixing_ratio_from_relative_humidity(p*units.hectopascal, t.metpy.quantify(), rh.metpy.quantify())
lim = z < 5000.
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\theta$ (K)')
ax1.set_title('2018-02-16')
# ax2.ylabel('Altitude (km)')
ax2.set_ylabel('')
ax2.set_ylim(0, 5)
ax2.set_xlabel(r'$r_v$ (g/kg)')
ax2.set_title('2018-02-16')
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# SONDE 1200 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180216.113100.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', label='SONDE 1131')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', label='SONDE 1131')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_ana(merra2_l_ena_ana, '2018-02-16T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1200', fillstyle='none')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1200', fillstyle='none')
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='red', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1200')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='red', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1200')
# SONDE 1800 UTC
sonde = xr.open_dataset(f'{obs_dir}/enasondewnpnC1.b1.20180216.173000.cdf')
z = sonde.alt
lim = z < 5000.
zlim = np.compress(lim, z)
tp = metpy.calc.potential_temperature(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'))
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', label='SONDE 1730')
rv = metpy.calc.mixing_ratio_from_relative_humidity(sonde.pres.metpy.quantify(), sonde.tdry.metpy.magnitude*metpy.units.units('degC'), sonde.rh.metpy.quantify())
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', label='SONDE 1730')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_ana(merra2_l_ena_ana, '2018-02-16T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1800', fillstyle='none')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=10, label='MERRA2 ANA 1800', fillstyle='none')
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
tp = np.compress(lim, tp)
ax1.plot(tp, zlim*1.0e-3, color='blue', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1800')
rv = np.compress(lim, rv)
ax2.plot(rv*1.0e3, zlim*1.0e-3, color='blue', linestyle='none', marker='+', markersize=10, label='MERRA2 ASM 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\theta$ (K)')
ax1.set_title('2018-02-16')
# ax2.ylabel('Altitude (km)')
ax2.set_ylabel('')
ax2.set_ylim(0, 5)
ax2.set_xlabel(r'$r_v$ (g/kg)')
ax2.set_title('2018-02-16')
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
fig = plt.figure(figsize=(6, 9))
ax1 = fig.add_axes((0.1, 0.1, 0.8, 0.8))
# 1200 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-16T12:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
w = np.compress(lim, w_era5.loc['2018-02-16T12:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
# FNL
z = fnl_l_ena.gh.loc['2018-02-16T12:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
w = np.compress(lim, w_fnl.loc['2018-02-16T12:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
w = np.compress(lim, w_merra2.loc['2018-02-16T12:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
# 1800 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-16T18:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
w = np.compress(lim, w_era5.loc['2018-02-16T18:00:00'])
ax1.plot(w*3600.0/100., zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
# FNL
z = fnl_l_ena.gh.loc['2018-02-16T18:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
w = np.compress(lim, w_fnl.loc['2018-02-16T18:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
w = np.compress(lim, w_merra2.loc['2018-02-16T18:00:00'])
ax1.plot(w*3600.0/100.0, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\omega$ (hPa/hr)')
ax1.set_title('2018-02-16')
plt.legend(fontsize=12)
plt.show()

---

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax1 = fig.add_axes((0.05, 0.1, 0.4, 0.8))
ax2 = fig.add_axes((0.5, 0.1, 0.4, 0.8))
# 1200 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-16T12:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
ta = np.compress(lim, ta_era5.loc['2018-02-16T12:00:00'])
ma = np.compress(lim, ma_era5.loc['2018-02-16T12:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='red', linestyle='none', marker='+', label='ERA5 1200')
# FNL
z = fnl_l_ena.gh.loc['2018-02-16T12:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_fnl.loc['2018-02-16T12:00:00'])
ma = np.compress(lim, ma_fnl.loc['2018-02-16T12:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='red', linestyle='none', marker='*', label='FNL 1200')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T12:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_merra2.loc['2018-02-16T12:00:00'])
ma = np.compress(lim, ma_merra2.loc['2018-02-16T12:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='red', linestyle='none', marker='o', markersize=5, label='MERRA2 1200')
# 1800 UTC
# ERA5
z_era5, p_era5, tp_era5, rv_era5 = extract_era5_profiles(era5_l_ena, '2018-02-16T18:00:00')
lim = z_era5 < 5000.*units.meter
zlim = np.compress(lim, z_era5)
ta = np.compress(lim, ta_era5.loc['2018-02-16T18:00:00'])
ma = np.compress(lim, ma_era5.loc['2018-02-16T18:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='blue', linestyle='none', marker='+', label='ERA5 1800')
# FNL
z = fnl_l_ena.gh.loc['2018-02-16T18:00:00']
lim = z < 5000.
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_fnl.loc['2018-02-16T18:00:00'])
ma = np.compress(lim, ma_fnl.loc['2018-02-16T18:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='blue', linestyle='none', marker='*', label='FNL 1800')
# MERRA2
z, p, tp, rv = extract_merra2_profiles_asm(merra2_l_ena, '2018-02-16T18:00:00')
lim = z < 5000.*units.meter
zlim = np.compress(lim, z)
ta = np.compress(lim, ta_merra2.loc['2018-02-16T18:00:00'])
ma = np.compress(lim, ma_merra2.loc['2018-02-16T18:00:00'])
ax1.plot(ta*3600.0, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax2.plot(ma*3600.0*1000.0, zlim*1.0e-3, color='blue', linestyle='none', marker='o', markersize=5, label='MERRA2 1800')
ax1.set_ylabel('Altitude (km)')
ax1.set_ylim(0, 5)
ax1.set_xlabel(r'$\theta$ horizontal advection (K/hr)')
ax1.set_title('2018-02-16')
ax2.set_ylabel('')
ax2.set_ylim(0, 5)
ax2.set_xlabel(r'$r_v$ horizontal advection (g/kg/hr)')
ax2.set_title('2018-02-16')
plt.legend(fontsize=12)
plt.legend(fontsize=12)
plt.show()